## Deliverable 2. Create a Customer Travel Destinations Map.

In [58]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [59]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Taltal,-25.4000,-70.4833,64.53,80,8,6.08,CL,clear sky
1,1,Yarada,17.6500,83.2667,82.36,69,31,10.00,IN,scattered clouds
2,2,Hermanus,-34.4187,19.2345,64.99,91,100,11.10,ZA,light rain
3,3,Altay,47.8667,88.1167,47.75,47,74,4.50,CN,broken clouds
4,4,Chokurdakh,70.6333,147.9167,10.83,90,66,13.04,RU,broken clouds


In [62]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [63]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temps_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]

In [64]:
# 4a. Determine if there are any empty rows.
temps_df.count()

City_ID        228
City           228
Lat            228
Lng            228
Max Temp       228
Humidity       228
Cloudiness     228
Wind Speed     228
Country        226
Description    228
dtype: int64

In [65]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = temps_df.dropna()
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
1,1,Yarada,17.6500,83.2667,82.36,69,31,10.00,IN,scattered clouds
9,9,Kapaa,22.0752,-159.3190,80.60,57,40,13.80,US,scattered clouds
27,27,Kavieng,-2.5744,150.7967,83.91,72,97,6.64,PG,overcast clouds
28,28,Puerto Baquerizo Moreno,-0.9000,-89.6000,76.93,88,100,4.12,EC,overcast clouds
31,31,Berbera,10.4396,45.0143,80.24,81,96,7.09,SO,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
689,689,Salinopolis,-0.6136,-47.3561,77.56,85,100,4.61,BR,overcast clouds
690,690,Sabang,5.8933,95.3214,82.76,74,100,15.39,ID,overcast clouds
694,694,Sambava,-14.2667,50.1667,76.15,89,90,15.43,MG,light rain
695,695,Gladstone,-23.8500,151.2500,77.00,50,75,6.91,AU,light rain


In [66]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Yarada,IN,82.36,scattered clouds,17.6500,83.2667,
9,Kapaa,US,80.60,scattered clouds,22.0752,-159.3190,
27,Kavieng,PG,83.91,overcast clouds,-2.5744,150.7967,
28,Puerto Baquerizo Moreno,EC,76.93,overcast clouds,-0.9000,-89.6000,
31,Berbera,SO,80.24,overcast clouds,10.4396,45.0143,
32,Sayla,IN,79.50,clear sky,22.5500,71.4667,
35,Rikitea,PF,77.92,few clouds,-23.1203,-134.9692,
45,Bambous Virieux,MU,78.80,light rain,-20.3428,57.7575,
48,Avarua,CK,84.20,light rain,-21.2078,-159.7750,
49,Deogarh,IN,77.22,clear sky,21.5333,84.7333,


In [71]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [72]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how = 'any', subset = ['Hotel Name'])

In [73]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [75]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [78]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…